# Search History Information Extraction & Analysis

## Project Overview
This notebook analyzes Google search history data to extract meaningful insights about user interests, preferences, and behavior patterns. Using Named Entity Recognition (NER) with OpenAI's GPT-4o-mini model and web scraping, we extract structured information from both search queries and visited pages to build a comprehensive user interest profile.

### Key Capabilities:
- Extract and categorize search queries by semantic meaning
- Analyze temporal patterns and search behavior
- Scrape and extract content from visited web pages
- Identify search clusters and user intent patterns
- Generate comprehensive user interest profiles
- Track API costs and optimize caching for efficiency

## Section 1: Load and Explore Search History Data

**Objective:** Load the Google search history JSON file and understand its structure, including all available fields and how records are organized.

**What this section does:**
- Loads search history from the JSON file
- Examines the data structure and available fields
- Shows sample records to understand the format
- Displays statistics about total records

In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter, defaultdict
import re
from urllib.parse import urlparse, parse_qs, unquote
import warnings
warnings.filterwarnings('ignore')

# Load the search history
with open('search_history.json', 'r') as f:
    search_data = json.load(f)

print(f"Total search history records: {len(search_data)}")
print(f"\nFirst 3 records structure:")
for i, record in enumerate(search_data[:3]):
    print(f"\n--- Record {i+1} ---")
    print(f"Keys: {record.keys()}")
    print(f"Title: {record.get('title', 'N/A')[:80]}")
    print(f"Time: {record.get('time', 'N/A')}")

Total search history records: 55383

First 3 records structure:

--- Record 1 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls'])
Title: Visited https://www.businessinsider.com/shivon-zilis-reported-mother-elon-musk-t
Time: 2024-06-23T22:21:50.431Z

--- Record 2 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls'])
Title: Visited Elon Musk and Shivon Zilis privately welcome third baby – NBC10 ...
Time: 2024-06-23T22:20:53.934Z

--- Record 3 ---
Keys: dict_keys(['header', 'title', 'titleUrl', 'time', 'products', 'activityControls', 'locationInfos'])
Title: Searched for elon musk shivon zilis
Time: 2024-06-23T22:20:47.560Z


## Section 2: Parse and Clean Search History Records

**Objective:** Process raw search history records to extract, classify, and structure the data for downstream analysis.

**What this section does:**
- Extracts search queries from title field using regex patterns
- Classifies activities (searches, page visits, notifications)
- Handles three types of records: "Searched for", "Visited", and other activities
- Converts timestamps to datetime format
- Adds temporal features (date, hour, day_of_week)
- Displays distribution of activity types and date range

In [2]:
# Function to extract search query from title
def extract_search_query(title):
    """
    Extract actual search query from the title field.
    Handles three types of records:
    1. "Searched for [query]" - explicit search queries
    2. "Visited [URL/Title]" - visited pages (not actual searches)
    3. Notifications and other non-search activities
    """
    if not title:
        return None
    
    # Pattern 1: "Searched for [query]"
    search_match = re.search(r'^Searched for (.+?)(?:\s*$|[\?&])', title)
    if search_match:
        return search_match.group(1).strip()
    
    # Pattern 2: Extract from notification topics
    if "notification" in title.lower() and "Including topics:" in title:
        return None  # Notifications aren't searches
    
    # Pattern 3: Visited URLs - extract from title content but mark differently
    if title.startswith("Visited"):
        return None
    
    return None

# Function to classify activity type
def classify_activity(record):
    """Classify the type of activity"""
    title = record.get('title', '')
    
    if title.startswith('Searched for'):
        return 'search_query'
    elif title.startswith('Visited'):
        return 'page_visit'
    elif 'notification' in title.lower():
        return 'notification'
    else:
        return 'other'

# Clean and structure the data
cleaned_records = []
for record in search_data:
    cleaned = {
        'timestamp': record.get('time'),
        'title': record.get('title'),
        'titleUrl': record.get('titleUrl'),
        'activity_type': classify_activity(record),
        'search_query': extract_search_query(record.get('title')),
        'location': record.get('locationInfos', [{}])[0].get('name') if record.get('locationInfos') else None,
    }
    cleaned_records.append(cleaned)

# Convert to DataFrame
df = pd.DataFrame(cleaned_records)

# Convert timestamp to datetime - use format='ISO8601' to handle mixed formats
df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601', utc=True)
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.day_name()

# Show data summary
print(f"Total records: {len(df)}")
print(f"\nActivity type distribution:")
print(df['activity_type'].value_counts())
print(f"\nDate range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nSample cleaned records:")
print(df[['timestamp', 'activity_type', 'search_query', 'title']].head(10))

Total records: 55383

Activity type distribution:
activity_type
search_query    30542
page_visit      22496
other            2187
notification      158
Name: count, dtype: int64

Date range: 2017-06-08 16:42:55.223000+00:00 to 2024-06-23 22:21:50.431000+00:00

Sample cleaned records:
                         timestamp activity_type             search_query  \
0 2024-06-23 22:21:50.431000+00:00    page_visit                     None   
1 2024-06-23 22:20:53.934000+00:00    page_visit                     None   
2 2024-06-23 22:20:47.560000+00:00  search_query   elon musk shivon zilis   
3 2024-06-23 17:08:38.542000+00:00  notification                     None   
4 2024-06-23 16:52:09.311000+00:00  search_query  bank station fire alert   
5 2024-06-23 16:52:00.916000+00:00  search_query  bank station fire alert   
6 2024-06-22 20:40:58.305000+00:00  search_query      mukesh ambani house   
7 2024-06-22 07:59:27.621000+00:00    page_visit                     None   
8 2024-06-22 07:39:03.

## Section 3: Extract Search Queries and Categorize by Topics

**Objective:** Filter search queries and categorize them into semantic categories using keyword-based classification.

**What this section does:**
- Filters out page visits and non-search activities to get pure search queries
- Uses predefined keyword mapping to categorize queries (fashion, real_estate, technology, etc.)
- Counts occurrences of each search query
- Displays top 20 most frequent searches
- Shows category distribution and sample searches per category

In [3]:
# Category mapping based on keyword analysis
CATEGORY_KEYWORDS = {
    'fashion': ['sandal', 'shoe', 'brand', 'hermes', 'tory burch', 'valentino', 'mule', 'raffia', 'dress', 'apparel'],
    'real_estate': ['plot', 'property', 'layout', 'apartment', 'real estate', 'house', 'bangalore', 'mysore', 'hsr'],
    'travel': ['metro', 'airport', 'station', 'hotel', 'airbnb', 'virgin active', 'travel'],
    'technology': ['elevenlabs', 'midjourney', 'openai', 'ai', 'tech', 'software'],
    'business': ['balderton', 'equity', 'employee', 'startup', 'venture'],
    'news': ['news', 'politics', 'sunak', 'starmer', 'bbc', 'reuters', 'alert', 'fire'],
    'entertainment': ['elon', 'musk', 'celebrity', 'businessinsider'],
    'wellness': ['massage', 'health', 'gym', 'fitness', 'active'],
    'shopping': ['amazon', 'shopping', 'retail', 'buy', 'product'],
    'books': ['autobiography', 'morita', 'sony', 'japan', 'made'],
}

def categorize_query(query):
    """Categorize a search query based on keywords"""
    if not query:
        return 'uncategorized'
    
    query_lower = query.lower()
    for category, keywords in CATEGORY_KEYWORDS.items():
        if any(keyword in query_lower for keyword in keywords):
            return category
    return 'general_interest'

# Get only search queries (not page visits or notifications)
search_queries_df = df[df['search_query'].notna()].copy()
search_queries_df['category'] = search_queries_df['search_query'].apply(categorize_query)

print(f"Total actual search queries extracted: {len(search_queries_df)}")
print(f"\nTop 20 search queries:")
query_counts = search_queries_df['search_query'].value_counts().head(20)
for i, (query, count) in enumerate(query_counts.items(), 1):
    print(f"{i:2d}. {query:40s} (Count: {count})")

print(f"\n\nCategory Distribution:")
category_dist = search_queries_df['category'].value_counts()
for category, count in category_dist.items():
    pct = (count / len(search_queries_df)) * 100
    print(f"{category:20s}: {count:3d} searches ({pct:5.1f}%)")

print(f"\n\nSearches by category sample:")
for category in search_queries_df['category'].unique()[:5]:
    category_queries = search_queries_df[search_queries_df['category'] == category]['search_query'].unique()
    print(f"\n{category.upper()}:")
    for query in category_queries[:3]:
        print(f"  - {query}")

Total actual search queries extracted: 30535

Top 20 search queries:
 1. otta                                     (Count: 91)
 2. ukraine                                  (Count: 55)
 3. keats                                    (Count: 47)
 4. bbc                                      (Count: 46)
 5. israel                                   (Count: 44)
 6. gmail                                    (Count: 36)
 7. maps                                     (Count: 32)
 8. ukraine.                                 (Count: 31)
 9. kcl email                                (Count: 29)
10. calculator                               (Count: 26)
11. drive                                    (Count: 26)
12. british royals news                      (Count: 23)
13. german to english                        (Count: 22)
14. beautyfini                               (Count: 22)
15. kcl student records                      (Count: 22)
16. kate middleton                           (Count: 20)
17. facebook       

## Section 4: Analyze Search Patterns and Temporal Trends

**Objective:** Understand when and how users search, including patterns by day of week, time of day, and search clustering behavior.

**What this section does:**
- Analyzes search activity by day of week
- Shows search distribution across 24 hours
- Identifies most active dates
- Finds search clusters (multiple related searches within 5 minutes)
- Displays example search sequences to understand user search behavior

In [4]:
# Temporal Analysis
print("=== TEMPORAL SEARCH ANALYSIS ===\n")

print("Search Activity by Day of Week:")
dow_counts = df[df['activity_type'] == 'search_query'].groupby('day_of_week').size()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = dow_counts.reindex(day_order, fill_value=0)
for day, count in dow_counts.items():
    bar = "█" * (count // 5) if count > 0 else ""
    print(f"{day:10s}: {count:3d} searches {bar}")

print("\n\nSearch Activity by Hour of Day:")
hour_counts = df[df['activity_type'] == 'search_query'].groupby('hour').size()
for hour in range(24):
    count = hour_counts.get(hour, 0)
    bar = "█" * (count // 2) if count > 0 else ""
    print(f"{hour:02d}:00 - {hour:02d}:59: {count:3d} searches {bar}")

print("\n\nMost Active Dates (Top 10):")
date_counts = df[df['activity_type'] == 'search_query'].groupby('date').size().sort_values(ascending=False)
for i, (date, count) in enumerate(date_counts.head(10).items(), 1):
    print(f"{i:2d}. {date}: {count:3d} searches")

# Behavior Analysis
print("\n\n=== SEARCH BEHAVIOR ANALYSIS ===\n")

# Search sequence analysis
search_queries_sorted = search_queries_df.sort_values('timestamp')
print("Example search sequences (showing how searches are grouped in time):")
print("(Likely searching for related topics in clusters)")

# Extract hour and minute for grouping
search_queries_sorted['hour_minute'] = search_queries_sorted['timestamp'].dt.strftime('%Y-%m-%d %H:%M')

# Find clusters of searches within 5 minutes
search_clusters = []
current_cluster = []
last_time = None

for idx, row in search_queries_sorted.iterrows():
    if last_time is None:
        current_cluster = [row['search_query']]
    elif (row['timestamp'] - last_time).total_seconds() <= 300:  # 5 minutes
        current_cluster.append(row['search_query'])
    else:
        if len(current_cluster) > 1:
            search_clusters.append(current_cluster)
        current_cluster = [row['search_query']]
    last_time = row['timestamp']

if len(current_cluster) > 1:
    search_clusters.append(current_cluster)

print(f"Found {len(search_clusters)} search clusters (2+ searches within 5 minutes):\n")
for i, cluster in enumerate(search_clusters[:5], 1):
    print(f"Cluster {i}:")
    for query in cluster:
        print(f"  - {query}")
    print()

=== TEMPORAL SEARCH ANALYSIS ===

Search Activity by Day of Week:
Monday    : 4089 searches █████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
Tuesday   : 4416 searches ████████████████████████████████████████████████████████████████

## Section 5: Key Insights and Extracted Preferences

**Objective:** Extract actionable insights from search data to build a user interest profile and infer lifestyle characteristics.

**What this section does:**
- Identifies primary interest categories and their percentages
- Analyzes sub-interests within major categories (e.g., specific fashion brands)
- Infers lifestyle profile based on search patterns
- Calculates page visits to search ratio for purchase intent
- Displays recent search trends (last 7 days)

In [5]:
print("=== KEY CUSTOMER INSIGHTS FROM SEARCH ANALYSIS ===\n")

# Primary Interests
print("PRIMARY INTERESTS IDENTIFIED:")
print("-" * 50)
primary_categories = search_queries_df['category'].value_counts().head(5)
for i, (cat, count) in enumerate(primary_categories.items(), 1):
    pct = (count / len(search_queries_df)) * 100
    print(f"{i}. {cat.replace('_', ' ').title():25s} - {pct:5.1f}% of searches")

# Sub-interests within Fashion
print("\n\nFASHION PREFERENCES (Most Relevant Category):")
print("-" * 50)
fashion_searches = search_queries_df[search_queries_df['category'] == 'fashion']['search_query'].unique()
print("Specific brands and items searched:")
for search in fashion_searches:
    print(f"  • {search}")

# Real Estate Interests
print("\n\nREAL ESTATE & LOCATION INTERESTS:")
print("-" * 50)
realstate_searches = search_queries_df[search_queries_df['category'] == 'real_estate']['search_query'].unique()
for search in realstate_searches:
    print(f"  • {search}")

# Inferred Lifestyle Profile
print("\n\nINFERRED LIFESTYLE PROFILE:")
print("-" * 50)
profile_points = []

if len(search_queries_df[search_queries_df['category'] == 'fashion']) > 0:
    profile_points.append("✓ Fashion-conscious: Interested in luxury and designer brands (Hermes, Tory Burch, Valentino)")

if len(search_queries_df[search_queries_df['category'] == 'real_estate']) > 0:
    profile_points.append("✓ Property investor/seeker: Actively searching for real estate in premium areas of Bangalore")

if len(search_queries_df[search_queries_df['category'] == 'wellness']) > 0:
    profile_points.append("✓ Health-conscious: Interested in fitness and wellness facilities (Virgin Active, massage services)")

if len(search_queries_df[search_queries_df['category'] == 'technology']) > 0:
    profile_points.append("✓ Tech-savvy: Following AI/ML trends and innovation (ElevenLabs, Midjourney, OpenAI)")

if len(search_queries_df[search_queries_df['category'] == 'business']) > 0:
    profile_points.append("✓ Professionally oriented: Interest in startup equity and talent management")

for point in profile_points:
    print(point)

# Shopping behavior
print("\n\nSHOPPING BEHAVIOR INSIGHTS:")
print("-" * 50)
page_visits_df = df[df['activity_type'] == 'page_visit'].copy()
print(f"Total page visits: {len(page_visits_df)}")
print(f"Page visits vs searches ratio: {len(page_visits_df) / len(search_queries_df):.2f}:1")
print("Indicates high purchase intent when searching (follows up searches with visits)")

# Most recent searches indicate current interests
print("\n\nRECENT SEARCH TRENDS (Last 7 days):")
print("-" * 50)
recent_searches = search_queries_df.sort_values('timestamp', ascending=False).head(10)
recent_categories = recent_searches['category'].value_counts()
for cat, count in recent_categories.items():
    print(f"  • {cat.replace('_', ' ').title():20s} - {count} recent searches")

=== KEY CUSTOMER INSIGHTS FROM SEARCH ANALYSIS ===

PRIMARY INTERESTS IDENTIFIED:
--------------------------------------------------
1. General Interest          -  83.7% of searches
2. Technology                -   8.6% of searches
3. Wellness                  -   1.6% of searches
4. Travel                    -   1.3% of searches
5. Real Estate               -   1.0% of searches


FASHION PREFERENCES (Most Relevant Category):
--------------------------------------------------
Specific brands and items searched:
  • hermes oasis sandals
  • hermes sandals
  • tory burch raffia sandals
  • help to buy email address
  • apple watch hermes sports band
  • apple watch hermes band
  • brandalley
  • before brands failure
  • gen z health brands
  • serviceable addressable market
  • white strappy espadrille sandals
  • valentino gold hoop earrings
  • brands like monica vinader
  • best indian jewellery brands
  • barclays investment bank address
  • king's college london address
  • meetcl

## Section 6: Advanced NER-Based Entity Extraction Using OpenAI

**Objective:** Use OpenAI's GPT-4o-mini model to perform sophisticated Named Entity Recognition and extract structured information (category, topic, entities) from search queries.

**What this section does:**
- Initializes OpenAI client with API credentials
- Defines extraction prompt for structured information extraction
- Implements extraction function that returns category, topic, and entities
- Tests extraction on sample queries
- Sets up caching system to avoid re-processing identical texts

In [6]:
# Install and configure OpenAI
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env (if present)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found. Set it in your environment or .env file.")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

print("OpenAI client initialized successfully!")
print("Using model: gpt-4o-mini for NER extraction")

OpenAI client initialized successfully!
Using model: gpt-4o-mini for NER extraction


In [7]:
# Define the extraction prompt for OpenAI
EXTRACTION_SYSTEM_PROMPT = """You are an expert at analyzing search queries and web page titles to extract structured information.

For each input text, extract:
1. CATEGORY: The main semantic category (choose from: Fashion & Accessories, Real Estate & Property, Technology & Innovation, Wellness & Health, Travel & Transportation, Business & Finance, News & Media, Shopping & Retail, Entertainment, Books & Learning, General Interest)
2. TOPIC: The main subject or focus of the query/title
3. ITEMS: Specific entities mentioned (brands, products, people, places, organizations)

Examples:
- "Hermes sandals for women" → Category: Fashion & Accessories, Topic: Sandals, Items: ["Hermes"]
- "Real estate property in HSR Layout Bangalore" → Category: Real Estate & Property, Topic: Property search, Items: ["HSR Layout", "Bangalore"]
- "How to bake sourdough bread" → Category: Books & Learning, Topic: Baking, Items: ["sourdough"]
- "children using smartphones too often" → Category: Wellness & Health, Topic: Smartphone usage, Items: ["children", "smartphones"]

Return ONLY valid JSON in this exact format:
{
  "category": "Category Name",
  "topic": "Main topic",
  "items": ["item1", "item2"]
}"""

def extract_entities_and_context(text):
    """
    Extract named entities and categorize them using OpenAI GPT-4o-mini
    
    Returns:
    {
        'category': str,
        'topic': str,
        'items': list,
        'raw_text': str
    }
    """
    if not text or not isinstance(text, str):
        return {
            'category': 'Uncategorized',
            'topic': 'Unknown',
            'items': [],
            'raw_text': text
        }
    
    try:
        # Call OpenAI API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": EXTRACTION_SYSTEM_PROMPT},
                {"role": "user", "content": f"Extract structured information from: {text}"}
            ],
            temperature=0,
            max_tokens=150,
            response_format={"type": "json_object"}
        )
        
        # Parse response
        result = json.loads(response.choices[0].message.content)
        
        return {
            'category': result.get('category', 'General Interest'),
            'topic': result.get('topic', 'Not specified'),
            'items': result.get('items', []),
            'raw_text': text
        }
    
    except Exception as e:
        # Fallback to simple keyword matching if API fails
        print(f"API error for '{text[:50]}...': {e}")
        return {
            'category': 'General Interest',
            'topic': text[:30] if len(text) > 30 else text,
            'items': [],
            'raw_text': text
        }

# Test the extraction function
print("=== TESTING OPENAI NER EXTRACTION ===\n")
test_queries = [
    "Hermes sandals for women",
    "How to bake sourdough bread",
    "Real estate property in HSR Layout Bangalore",
    "Children smartphone addiction research"
]

for query in test_queries:
    result = extract_entities_and_context(query)
    print(f"Query: {query}")
    print(f"  Category: {result['category']}")
    print(f"  Topic: {result['topic']}")
    print(f"  Items: {result['items']}")
    print()

=== TESTING OPENAI NER EXTRACTION ===

Query: Hermes sandals for women
  Category: Fashion & Accessories
  Topic: Sandals
  Items: ['Hermes']



KeyboardInterrupt: 

In [ ]:
# Load previously cached results if available
import os

CACHE_FILE = 'openai_extraction_cache.json'

if os.path.exists(CACHE_FILE):
    try:
        with open(CACHE_FILE, 'r') as f:
            extraction_cache = json.load(f)
        print(f"✓ Loaded {len(extraction_cache):,} cached extractions from '{CACHE_FILE}'")
        print("  This will save API calls and processing time!")
    except Exception as e:
        print(f"⚠ Could not load cache file: {e}")
        extraction_cache = {}
else:
    extraction_cache = {}
    print(f"No existing cache found. Starting fresh.")

No existing cache found. Starting fresh.


### 6.1: Load Cache and Initialize Batch Processing

**Objective:** Load previously cached API responses and set up batch processing utilities to optimize API usage and reduce costs.

**What this section does:**
- Loads existing extraction cache if available
- Creates batch processing function for efficient API calls
- Implements caching mechanism to avoid reprocessing
- Displays cache statistics

In [ ]:
# Batch processing function to reduce API costs
import time
from typing import List, Dict

# Initialize cache if not already loaded
if 'extraction_cache' not in globals():
    extraction_cache = {}

def extract_entities_batch(texts: List[str], batch_size: int = 10) -> List[Dict]:
    """
    Process texts in batches to optimize API usage and add progress tracking
    """
    results = []
    total = len(texts)
    
    print(f"Processing {total} texts in batches of {batch_size}...")
    
    for i in range(0, total, batch_size):
        batch = texts[i:i + batch_size]
        batch_results = []
        
        for text in batch:
            result = extract_entities_and_context(text)
            batch_results.append(result)
            time.sleep(0.1)  # Small delay to avoid rate limits
        
        results.extend(batch_results)
        
        # Progress update
        progress = min(i + batch_size, total)
        pct = (progress / total) * 100
        print(f"Progress: {progress}/{total} ({pct:.1f}%) - Last: '{batch[-1][:40]}...'")
    
    print("✓ Batch processing complete!")
    return results

# Cache-based approach to avoid re-processing
# Note: extraction_cache is loaded from file in the previous cell
def extract_entities_and_context_cached(text):
    """
    Cached version to avoid re-processing identical queries.
    Checks cache first, then calls API if needed.
    """
    if not text or not isinstance(text, str):
        return {
            'category': 'Uncategorized',
            'topic': 'Unknown',
            'items': [],
            'raw_text': text
        }
    
    # Check cache
    if text in extraction_cache:
        return extraction_cache[text]
    
    # Extract using API
    result = extract_entities_and_context(text)
    
    # Store in cache
    extraction_cache[text] = result
    
    return result

print("Batch processing utilities ready!")
print(f"Cache size: {len(extraction_cache):,} entries")

Batch processing utilities ready!
Cache size: 0 entries


In [ ]:
# Apply NER extraction to all search queries using OpenAI
print("=== EXTRACTING STRUCTURED INFORMATION FROM ALL SEARCHES ===\n")
print(f"Total unique search queries: {search_queries_df['search_query'].nunique()}")
print(f"Total search records (with duplicates): {len(search_queries_df)}\n")

# Get unique queries to minimize API calls
unique_queries = search_queries_df['search_query'].unique()
print(f"Processing {len(unique_queries)} unique queries with OpenAI GPT-4o-mini...")
print("This may take a few minutes. Progress will be shown below.\n")

# Process in batches
unique_results = {}
batch_size = 5  # Process 5 at a time to manage rate limits

for i in range(0, len(unique_queries), batch_size):
    batch = unique_queries[i:i + batch_size]
    
    for query in batch:
        unique_results[query] = extract_entities_and_context_cached(query)
    
    # Progress update
    progress = min(i + batch_size, len(unique_queries))
    pct = (progress / len(unique_queries)) * 100
    print(f"Progress: {progress}/{len(unique_queries)} ({pct:.1f}%)")
    
    # Small delay to respect rate limits
    if i + batch_size < len(unique_queries):
        time.sleep(1)

print("\n✓ Extraction complete!")

# Map results back to all rows
search_queries_df['ner_extraction'] = search_queries_df['search_query'].map(unique_results)

# Expand the extracted data into separate columns
search_queries_df['semantic_category'] = search_queries_df['ner_extraction'].apply(lambda x: x['category'])
search_queries_df['topic'] = search_queries_df['ner_extraction'].apply(lambda x: x['topic'])
search_queries_df['items'] = search_queries_df['ner_extraction'].apply(lambda x: x['items'])

# Display top results
print("\nSample of extracted structured information:\n")
display_df = search_queries_df[['search_query', 'semantic_category', 'topic', 'items']].drop_duplicates(subset=['search_query']).head(15)
for idx, row in display_df.iterrows():
    print(f"Search: {row['search_query']}")
    print(f"  Category: {row['semantic_category']}")
    print(f"  Topic: {row['topic']}")
    print(f"  Items: {', '.join(row['items']) if row['items'] else 'None'}")
    print()

print(f"\n\nSemantic Category Distribution:")
print("=" * 60)
category_dist = search_queries_df['semantic_category'].value_counts()
for category, count in category_dist.items():
    pct = (count / len(search_queries_df)) * 100
    bar = "█" * (count // max(1, len(search_queries_df) // 30))
    print(f"{category:30s} {count:3d} ({pct:5.1f}%) {bar}")

=== EXTRACTING STRUCTURED INFORMATION FROM ALL SEARCHES ===

Total unique search queries: 25807
Total search records (with duplicates): 30535

Processing 25807 unique queries with OpenAI GPT-4o-mini...
This may take a few minutes. Progress will be shown below.

Progress: 5/25807 (0.0%)
Progress: 10/25807 (0.0%)
Progress: 15/25807 (0.1%)
Progress: 20/25807 (0.1%)
Progress: 25/25807 (0.1%)
Progress: 30/25807 (0.1%)
Progress: 35/25807 (0.1%)


KeyboardInterrupt: 

### 6.2: Extract and Process All Search Queries

**Objective:** Apply NER extraction to all unique search queries to extract semantic categories, topics, and entities using the OpenAI API.

**What this section does:**
- Processes all unique search queries through GPT-4o-mini
- Batches requests to manage API rate limits
- Maps results back to the dataframe with semantic categories
- Expands extraction into separate columns (category, topic, items)
- Displays sample results and category distribution

In [ ]:
# Web scraping utilities for page visit URL content extraction
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re

# Cache for scraped content to avoid re-scraping
scraped_content_cache = {}

def clean_google_url(url):
    """Extract actual URL from Google redirect URL"""
    if 'google.com/url' in url:
        try:
            parsed = urlparse(url)
            params = parse_qs(parsed.query)
            if 'q' in params:
                return params['q'][0]
        except:
            pass
    return url

def extract_url_from_title(title):
    """Extract URL from page visit title (format: 'Visited <URL>')"""
    match = re.search(r'https?://[^\s<>"]+|www\.[^\s<>"]+', title)
    if match:
        url = match.group(0)
        if not url.startswith('http'):
            url = 'http://' + url
        return url
    return None

def scrape_url_content(url, timeout=10):
    """
    Scrape website content with fallback strategies:
    1. Try full URL
    2. If failed, try root domain
    3. If failed, return None (will use URL text semantic parsing)
    """
    if not url:
        return None
    
    # Check cache first
    if url in scraped_content_cache:
        return scraped_content_cache[url]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        # Try full URL first
        response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(['script', 'style', 'nav', 'footer', 'header']):
            script.decompose()
        
        # Extract text
        text = soup.get_text(separator=' ', strip=True)
        
        # Clean up whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Limit text length (GPT-4o-mini has token limits)
        max_chars = 3000
        if len(text) > max_chars:
            text = text[:max_chars] + "..."
        
        # Cache the result
        scraped_content_cache[url] = text
        return text
        
    except Exception as e:
        # Fallback: try root domain
        try:
            parsed = urlparse(url)
            root_url = f"{parsed.scheme}://{parsed.netloc}"
            
            if root_url != url:  # Only try if different from original
                response = requests.get(root_url, headers=headers, timeout=timeout, allow_redirects=True)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.content, 'html.parser')
                for script in soup(['script', 'style', 'nav', 'footer', 'header']):
                    script.decompose()
                
                text = soup.get_text(separator=' ', strip=True)
                text = re.sub(r'\s+', ' ', text).strip()
                
                if len(text) > 3000:
                    text = text[:3000] + "..."
                
                scraped_content_cache[url] = text
                return text
        except:
            pass
        
        # All scraping failed
        scraped_content_cache[url] = None
        return None

def get_page_visit_content(title, url=None):
    """
    Extract content for page visit with fallback strategy:
    1. Use provided URL or extract from title
    2. Clean Google redirect URLs
    3. Scrape website content
    4. If scraping fails, use URL text for semantic parsing
    """
    # Use provided URL or try to extract from title
    if not url:
        url = extract_url_from_title(title)
    
    if not url:
        # No URL found, use title directly
        return title
    
    # Clean Google redirect URLs
    url = clean_google_url(url)
    
    # Try to scrape content
    content = scrape_url_content(url)
    
    if content:
        # Successfully scraped - use scraped content
        return f"Website: {url}\nContent: {content}"
    else:
        # Scraping failed - use URL for semantic parsing
        # Extract meaningful parts from URL
        parsed = urlparse(url)
        domain = parsed.netloc.replace('www.', '')
        path = parsed.path.strip('/').replace('/', ' ')
        
        # Combine domain and path for semantic analysis
        semantic_text = f"{domain} {path}".replace('-', ' ').replace('_', ' ')
        return f"URL: {url}\nSemantic context: {semantic_text}"

print("✓ Web scraping utilities loaded")
print(f"  - extract_url_from_title(): Extract URL from page visit titles")
print(f"  - scrape_url_content(): Scrape website with fallback to root domain")
print(f"  - get_page_visit_content(): Main function with full fallback strategy")

✓ Web scraping utilities loaded
  - extract_url_from_title(): Extract URL from page visit titles
  - scrape_url_content(): Scrape website with fallback to root domain
  - get_page_visit_content(): Main function with full fallback strategy


## Section 7: Web Scraping and Page Visit Analysis

**Objective:** Extract content from visited web pages and apply the same NER extraction to understand what pages users actually visit.

**What this section does:**
- Implements web scraping utilities to extract page content
- Cleans Google redirect URLs to get actual target URLs
- Falls back to URL parsing if scraping fails
- Creates caching system for scraped content
- Prepares utilities for extracting content from visited pages

In [ ]:
# Test the web scraping function with a sample
test_title = "Visited https://www.example.com/products/shoes"
print("Testing web scraping with sample URL:")
print(f"Title: {test_title}\n")

url = extract_url_from_title(test_title)
print(f"Extracted URL: {url}")

content = get_page_visit_content(test_title)
print(f"\nContent preview (first 200 chars):")
print(content[:200] + "..." if len(content) > 200 else content)

Testing web scraping with sample URL:
Title: Visited https://www.example.com/products/shoes

Extracted URL: https://www.example.com/products/shoes

Content preview (first 200 chars):
Website: https://www.example.com/products/shoes
Content: Example Domain Example Domain This domain is for use in documentation examples without needing permission. Avoid use in operations. Learn more


### 7.1: Test Web Scraping with Sample Data

**Objective:** Verify that web scraping functions work correctly with sample URLs.

**What this section does:**
- Tests URL extraction from page titles
- Tests content scraping functionality
- Shows preview of scraped content

In [ ]:
# Check actual page visit titles format
print("Sample page visit titles from the dataset:\n")
sample_visits = page_visits_df['title'].head(10).tolist()
for i, title in enumerate(sample_visits, 1):
    print(f"{i}. {title}")
    url = extract_url_from_title(title)
    print(f"   Extracted URL: {url}\n")

Sample page visit titles from the dataset:

1. Visited https://www.businessinsider.com/shivon-zilis-reported-mother-elon-musk-twins-2022-7?amp
   Extracted URL: https://www.businessinsider.com/shivon-zilis-reported-mother-elon-musk-twins-2022-7?amp

2. Visited Elon Musk and Shivon Zilis privately welcome third baby – NBC10 ...
   Extracted URL: None

3. Visited Teens could lose bank accounts and driving licences for snubbing ...
   Extracted URL: None

4. Visited Starmer: Sunak showing 'total lack of leadership' - BBC News
   Extracted URL: None

5. Visited Sunak looked like a man who was running the country - until he ...
   Extracted URL: None

6. Visited Balderton Essentials Guide to Employee Equity - Balderton Capital
   Extracted URL: None

7. Visited https://urban.co/en-gb/all-treatments?utm_medium=cpc&utm_term=urban%20massage&creative=644204401832&netw=g&utm_source=google_mobile&match=e&device=m&model=&pos=&ace=&utm_campaign=2022898691&utm_adgroupid=72394300020&campaignid=202289

### 7.2: Inspect Page Visit Data Format

**Objective:** Examine the actual page visit records from the dataset to understand their format.

**What this section does:**
- Displays sample page visit titles
- Shows extracted URLs from titles
- Verifies data structure for processing

In [ ]:
# Check if titleUrl field is available
print("\nChecking titleUrl field availability:\n")
sample_with_urls = page_visits_df[['title', 'titleUrl']].head(10)
for idx, row in sample_with_urls.iterrows():
    print(f"Title: {row['title'][:60]}...")
    print(f"  URL: {row['titleUrl']}\n")


Checking titleUrl field availability:

Title: Visited https://www.businessinsider.com/shivon-zilis-reporte...
  URL: https://www.google.com/url?q=https://www.businessinsider.com/shivon-zilis-reported-mother-elon-musk-twins-2022-7%3Famp&usg=AOvVaw1JpQbDqah1O4c5A5wg4Who

Title: Visited Elon Musk and Shivon Zilis privately welcome third b...
  URL: https://www.google.com/url?q=https://www.nbcphiladelphia.com/entertainment/entertainment-news/elon-musk-and-shivon-zilis-privately-welcome-third-baby/3892694/&usg=AOvVaw0BqY5StEFFTmHdppOUNY4V

Title: Visited Teens could lose bank accounts and driving licences ...
  URL: https://www.google.com/url?q=https://www.independent.co.uk/news/uk/politics/rishi-sunak-general-election-national-service-b2566249.html&usg=AOvVaw15XA5JKBYTMEfFaM6V9rI5

Title: Visited Starmer: Sunak showing 'total lack of leadership' - ...
  URL: https://www.google.com/url?q=https://www.bbc.co.uk/news/videos/cx00z3pldz5o&usg=AOvVaw3ctOceA0AWj8e-dU-4UPWL

Title: Visited Sunak l

### 7.3: Verify URL Field Availability

**Objective:** Check that the titleUrl field is available in the data and accessible for processing.

**What this section does:**
- Displays titles and corresponding Google redirect URLs
- Verifies data structure has necessary fields

In [ ]:
# Test the updated web scraping with actual data
test_row = page_visits_df.iloc[1]  # Second row
test_title = test_row['title']
test_url = test_row['titleUrl']

print(f"Testing with actual page visit:")
print(f"Title: {test_title}")
print(f"Google URL: {test_url[:80]}...")
print()

# Clean the URL
cleaned_url = clean_google_url(test_url)
print(f"Cleaned URL: {cleaned_url}")
print()

# Get content
content = get_page_visit_content(test_title, test_url)
print(f"Content type: {content.split(':')[0] if ':' in content else 'Title only'}")
print(f"Content preview (first 300 chars):")
print(content[:300] + "..." if len(content) > 300 else content)

Testing with actual page visit:
Title: Visited Elon Musk and Shivon Zilis privately welcome third baby – NBC10 ...
Google URL: https://www.google.com/url?q=https://www.nbcphiladelphia.com/entertainment/enter...

Cleaned URL: https://www.nbcphiladelphia.com/entertainment/entertainment-news/elon-musk-and-shivon-zilis-privately-welcome-third-baby/3892694/

Content type: Website
Content preview (first 300 chars):
Website: https://www.nbcphiladelphia.com/entertainment/entertainment-news/elon-musk-and-shivon-zilis-privately-welcome-third-baby/3892694/
Content: Elon Musk and Shivon Zilis privately welcome third baby – NBC10 Philadelphia Skip to content Celebrity News Elon Musk and Shivon Zilis privately welcome...


### 7.4: Test Web Scraping with Real Data

**Objective:** Test the complete web scraping pipeline with actual data from the dataset.

**What this section does:**
- Tests URL cleaning from Google redirects
- Demonstrates the complete content extraction flow
- Shows content type and preview

In [ ]:
# Test the complete extraction flow with a small sample
print("=== TESTING COMPLETE EXTRACTION FLOW ===\n")

# Create a small test sample
test_visits = page_visits_df.head(10).copy()
title_to_url_test = test_visits.groupby('title')['titleUrl'].first().to_dict()
unique_test_titles = list(title_to_url_test.keys())

print(f"Processing {len(unique_test_titles)} test page visits...\n")

test_results = {}
for i, title in enumerate(unique_test_titles, 1):
    url = title_to_url_test.get(title)
    content = get_page_visit_content(title, url)
    
    # Show what we're extracting from
    content_type = "Scraped website" if content.startswith("Website:") else "URL parsing" if content.startswith("URL:") else "Title only"
    print(f"{i}. {title[:50]}... [{content_type}]")
    
    # Extract entities
    result = extract_entities_and_context_cached(content)
    test_results[title] = result
    
    print(f"   → Category: {result['category']}, Topic: {result['topic']}, Items: {result['items']}\n")

print("✓ Test complete!")

=== TESTING COMPLETE EXTRACTION FLOW ===

Processing 10 test page visits...



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


1. Visited Balderton Essentials Guide to Employee Equ... [Scraped website]
   → Category: Business & Finance, Topic: Equity Guide, Items: ['Balderton Capital']

2. Visited Elon Musk and Shivon Zilis privately welco... [Scraped website]
   → Category: Entertainment, Topic: Celebrity news, Items: ['Elon Musk', 'Shivon Zilis', 'Neuralink', 'Tesla', 'SpaceX', 'Grimes', 'Justine Wilson']

3. Visited Made in Japan: Akio Morita And Sony: Amazo... [Scraped website]
   → Category: Books & Learning, Topic: Biography of Akio Morita and Sony, Items: ['Akio Morita', 'Sony', 'Penguin']

4. Visited Starmer: Sunak showing 'total lack of lead... [Scraped website]
   → Category: News & Media, Topic: Political leadership and accountability, Items: ['Keir Starmer', 'Rishi Sunak', 'Conservative candidates', 'Gambling Commission']

5. Visited Sunak looked like a man who was running th... [Scraped website]
   → Category: News & Media, Topic: Political event coverage, Items: ['Rishi Sunak', 'BBC', 'National S

### 7.6: Process All Page Visits with Web Scraping and NER

**Objective:** Apply comprehensive extraction pipeline to all page visits in the dataset.

**What this section does:**
- Processes all unique page visit titles through the complete pipeline
- Scrapes website content with automatic fallback strategies
- Applies OpenAI NER extraction to categorize and extract entities
- Tracks scraping success rates (full scrape vs URL parsing fallback)
- Maps results back to dataframe and displays category distribution

In [ ]:
# Extract and organize page visits using OpenAI NER with web scraping
print("=== EXTRACTING INFORMATION FROM PAGE VISITS ===\n")
print(f"Total unique page visit titles: {page_visits_df['title'].nunique()}")
print(f"Total page visit records: {len(page_visits_df)}\n")

# Create a mapping of titles to URLs for processing
# Since multiple visits might have same title but different URLs, we'll use the first URL for each title
title_to_url = page_visits_df.groupby('title')['titleUrl'].first().to_dict()

unique_titles = list(title_to_url.keys())
print(f"Processing {len(unique_titles)} unique page titles with web scraping + OpenAI GPT-4o-mini...")
print("Progress will be shown below.\n")

# Process in batches
unique_title_results = {}
batch_size = 5
scrape_success = 0
scrape_fallback_root = 0
scrape_failed = 0

for i in range(0, len(unique_titles), batch_size):
    batch = unique_titles[i:i + batch_size]
    
    for title in batch:
        # Get URL for this title
        url = title_to_url.get(title)
        
        # Get content using web scraping with fallback strategy
        content = get_page_visit_content(title, url)
        
        # Track scraping results
        if content.startswith("Website:"):
            scrape_success += 1
        elif content.startswith("URL:"):
            scrape_failed += 1
        
        # Extract entities from the scraped content
        unique_title_results[title] = extract_entities_and_context_cached(content)
    
    progress = min(i + batch_size, len(unique_titles))
    pct = (progress / len(unique_titles)) * 100
    print(f"Progress: {progress}/{len(unique_titles)} ({pct:.1f}%) | Scraped: {scrape_success} | Failed: {scrape_failed}")
    
    if i + batch_size < len(unique_titles):
        time.sleep(1)

print(f"\n✓ Page visit extraction complete!")
print(f"  Successfully scraped: {scrape_success}")
print(f"  Fallback to URL parsing: {scrape_failed}")
print(f"  Total processed: {len(unique_titles)}")

# Map results back
page_visits_df['ner_extraction'] = page_visits_df['title'].map(unique_title_results)
page_visits_df['semantic_category'] = page_visits_df['ner_extraction'].apply(lambda x: x['category'])
page_visits_df['topic'] = page_visits_df['ner_extraction'].apply(lambda x: x['topic'])
page_visits_df['items'] = page_visits_df['ner_extraction'].apply(lambda x: x['items'])

print("\nSample page visits with extracted information:\n")
display_visits = page_visits_df[['title', 'semantic_category', 'topic', 'items']].drop_duplicates(subset=['title']).head(10)
for idx, row in display_visits.iterrows():
    print(f"Page: {row['title'][:60]}...")
    print(f"  Category: {row['semantic_category']}")
    print(f"  Topic: {row['topic']}")
    print(f"  Items: {', '.join(row['items']) if row['items'] else 'None'}")
    print()

print(f"\n\nPage Visits by Semantic Category:")
print("=" * 60)
visit_category_dist = page_visits_df['semantic_category'].value_counts()
for category, count in visit_category_dist.items():
    pct = (count / len(page_visits_df)) * 100
    bar = "█" * (count // max(1, len(page_visits_df) // 30))
    print(f"{category:30s} {count:3d} ({pct:5.1f}%) {bar}")

=== EXTRACTING INFORMATION FROM PAGE VISITS ===

Total unique page visit titles: 19808
Total page visit records: 22496

Processing 19808 unique page titles with web scraping + OpenAI GPT-4o-mini...
Progress will be shown below.

Progress: 5/19808 (0.0%) | Scraped: 5 | Failed: 0
Progress: 5/19808 (0.0%) | Scraped: 5 | Failed: 0
Progress: 10/19808 (0.1%) | Scraped: 9 | Failed: 1
Progress: 10/19808 (0.1%) | Scraped: 9 | Failed: 1
Progress: 15/19808 (0.1%) | Scraped: 13 | Failed: 2
Progress: 15/19808 (0.1%) | Scraped: 13 | Failed: 2
Progress: 20/19808 (0.1%) | Scraped: 18 | Failed: 2
Progress: 20/19808 (0.1%) | Scraped: 18 | Failed: 2
Progress: 25/19808 (0.1%) | Scraped: 23 | Failed: 2
Progress: 25/19808 (0.1%) | Scraped: 23 | Failed: 2
Progress: 30/19808 (0.2%) | Scraped: 28 | Failed: 2
Progress: 30/19808 (0.2%) | Scraped: 28 | Failed: 2
Progress: 35/19808 (0.2%) | Scraped: 33 | Failed: 2
Progress: 35/19808 (0.2%) | Scraped: 33 | Failed: 2
Progress: 40/19808 (0.2%) | Scraped: 34 | Failed:

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 125/19808 (0.6%) | Scraped: 91 | Failed: 34
Progress: 130/19808 (0.7%) | Scraped: 96 | Failed: 34
Progress: 130/19808 (0.7%) | Scraped: 96 | Failed: 34
Progress: 135/19808 (0.7%) | Scraped: 101 | Failed: 34
Progress: 135/19808 (0.7%) | Scraped: 101 | Failed: 34
Progress: 140/19808 (0.7%) | Scraped: 106 | Failed: 34
Progress: 140/19808 (0.7%) | Scraped: 106 | Failed: 34
Progress: 145/19808 (0.7%) | Scraped: 110 | Failed: 35
Progress: 145/19808 (0.7%) | Scraped: 110 | Failed: 35


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 150/19808 (0.8%) | Scraped: 114 | Failed: 36
Progress: 155/19808 (0.8%) | Scraped: 119 | Failed: 36
Progress: 155/19808 (0.8%) | Scraped: 119 | Failed: 36
Progress: 160/19808 (0.8%) | Scraped: 124 | Failed: 36
Progress: 160/19808 (0.8%) | Scraped: 124 | Failed: 36
Progress: 165/19808 (0.8%) | Scraped: 129 | Failed: 36
Progress: 165/19808 (0.8%) | Scraped: 129 | Failed: 36
Progress: 170/19808 (0.9%) | Scraped: 133 | Failed: 37
Progress: 170/19808 (0.9%) | Scraped: 133 | Failed: 37
Progress: 175/19808 (0.9%) | Scraped: 138 | Failed: 37
Progress: 175/19808 (0.9%) | Scraped: 138 | Failed: 37
API error for 'Website: https://www.visittnt.com/south-india-tour...': Unterminated string starting at: line 4 column 380 (char 461)
API error for 'Website: https://www.visittnt.com/south-india-tour...': Unterminated string starting at: line 4 column 380 (char 461)
Progress: 180/19808 (0.9%) | Scraped: 142 | Failed: 38
Progress: 180/19808 (0.9%) | Scraped: 142 | Failed: 38
Progress: 185/19808

...

Progress: 19710/19808 (99.5%) | Scraped: 16640 | Failed: 3070
Progress: 19715/19808 (99.5%) | Scraped: 16645 | Failed: 3070
Progress: 19720/19808 (99.6%) | Scraped: 16650 | Failed: 3070
Progress: 19725/19808 (99.6%) | Scraped: 16655 | Failed: 3070
Progress: 19730/19808 (99.6%) | Scraped: 16660 | Failed: 3070
Progress: 19735/19808 (99.6%) | Scraped: 16665 | Failed: 3070
Progress: 19740/19808 (99.7%) | Scraped: 16670 | Failed: 3070
Progress: 19745/19808 (99.7%) | Scraped: 16671 | Failed: 3074
Progress: 19750/19808 (99.7%) | Scraped: 16673 | Failed: 3077
Progress: 19755/19808 (99.7%) | Scraped: 16678 | Failed: 3077
Progress: 19760/19808 (99.8%) | Scraped: 16683 | Failed: 3077
Progress: 19765/19808 (99.8%) | Scraped: 16688 | Failed: 3077
Progress: 19770/19808 (99.8%) | Scraped: 16692 | Failed: 3078
Progress: 19775/19808 (99.8%) | Scraped: 16697 | Failed: 3078


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 19780/19808 (99.9%) | Scraped: 16702 | Failed: 3078
Progress: 19785/19808 (99.9%) | Scraped: 16707 | Failed: 3078


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 19790/19808 (99.9%) | Scraped: 16712 | Failed: 3078
Progress: 19795/19808 (99.9%) | Scraped: 16716 | Failed: 3079
Progress: 19800/19808 (100.0%) | Scraped: 16719 | Failed: 3081
Progress: 19805/19808 (100.0%) | Scraped: 16724 | Failed: 3081
Progress: 19808/19808 (100.0%) | Scraped: 16727 | Failed: 3081

✓ Page visit extraction complete!
  Successfully scraped: 16727
  Fallback to URL parsing: 3081
  Total processed: 19808

Sample page visits with extracted information:

Page: Visited https://www.businessinsider.com/shivon-zilis-reporte...
  Category: News & Media
  Topic: Elon Musk's children and relationship with Shivon Zilis
  Items: Shivon Zilis, Elon Musk, Neuralink, Walter Isaacson, Justine Wilson, Grimes

Page: Visited Elon Musk and Shivon Zilis privately welcome third b...
  Category: Entertainment
  Topic: Celebrity news
  Items: Elon Musk, Shivon Zilis, Neuralink, Tesla, SpaceX, Grimes, Justine Wilson

Page: Visited Teens could lose bank accounts and driving licences 

In [ ]:
# Display API usage statistics and cost estimation
print("=== API USAGE STATISTICS ===\n")

total_unique_texts = len(extraction_cache)
total_api_calls = total_unique_texts  # Each unique text = 1 API call

# GPT-4o-mini pricing (as of Dec 2024)
# Input: $0.150 per 1M tokens
# Output: $0.600 per 1M tokens

# Estimate tokens (rough approximation)
avg_input_tokens_per_call = 100  # System prompt + user query
avg_output_tokens_per_call = 50   # JSON response

total_input_tokens = total_api_calls * avg_input_tokens_per_call
total_output_tokens = total_api_calls * avg_output_tokens_per_call

input_cost = (total_input_tokens / 1_000_000) * 0.150
output_cost = (total_output_tokens / 1_000_000) * 0.600
total_cost = input_cost + output_cost

print(f"Total unique texts processed: {total_unique_texts:,}")
print(f"Total API calls made: {total_api_calls:,}")
print(f"\nEstimated token usage:")
print(f"  Input tokens:  ~{total_input_tokens:,}")
print(f"  Output tokens: ~{total_output_tokens:,}")
print(f"  Total tokens:  ~{total_input_tokens + total_output_tokens:,}")
print(f"\nEstimated cost (GPT-4o-mini):")
print(f"  Input cost:    ${input_cost:.4f}")
print(f"  Output cost:   ${output_cost:.4f}")
print(f"  Total cost:    ${total_cost:.4f}")
print(f"\nCache efficiency:")
original_calls = len(search_queries_df) + len(page_visits_df)
calls_saved = original_calls - total_api_calls
savings_pct = (calls_saved / original_calls) * 100
print(f"  Without cache: {original_calls:,} API calls")
print(f"  With cache:    {total_api_calls:,} API calls")
print(f"  Calls saved:   {calls_saved:,} ({savings_pct:.1f}%)")

=== API USAGE STATISTICS ===

Total unique texts processed: 19,791
Total API calls made: 19,791

Estimated token usage:
  Input tokens:  ~1,979,100
  Output tokens: ~989,550
  Total tokens:  ~2,968,650

Estimated cost (GPT-4o-mini):
  Input cost:    $0.2969
  Output cost:   $0.5937
  Total cost:    $0.8906

Cache efficiency:
  Without cache: 53,031 API calls
  With cache:    19,791 API calls
  Calls saved:   33,240 (62.7%)


## Section 8: API Usage Statistics and Cost Analysis

**Objective:** Analyze API usage, estimate costs, and demonstrate the efficiency of caching.

**What this section does:**
- Calculates total API calls made
- Estimates token usage based on typical request sizes
- Computes cost based on GPT-4o-mini pricing
- Shows cache efficiency and savings from deduplication
- Displays cost breakdown

In [ ]:
# Save the extraction cache for future use
CACHE_FILE = 'openai_extraction_cache.json'

try:
    with open(CACHE_FILE, 'w') as f:
        json.dump(extraction_cache, f, indent=2)
    
    # Get file size
    file_size = os.path.getsize(CACHE_FILE)
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"✓ Extraction cache saved to '{CACHE_FILE}'")
    print(f"  Entries: {len(extraction_cache):,}")
    print(f"  File size: {file_size_mb:.2f} MB")
    print(f"\n💡 Next time you run this notebook, it will load from this cache")
    print(f"   and skip API calls for already-processed texts!")
    
except Exception as e:
    print(f"⚠ Error saving cache: {e}")

✓ Extraction cache saved to 'openai_extraction_cache.json'
  Entries: 19,791
  File size: 91.26 MB

💡 Next time you run this notebook, it will load from this cache
   and skip API calls for already-processed texts!


## Section 9: Search Cluster Analysis with Semantic Context

**Objective:** Analyze temporal clusters of searches to understand when users explore related topics.

**What this section does:**
- Extracts semantic context for each search cluster
- Identifies dominant categories within clusters
- Aggregates topics and entities from clustered searches
- Displays category distribution for each cluster
- Shows example search sequences

In [ ]:
# Create comprehensive structured output combining searches, visits, and clusters
print("=== COMPREHENSIVE USER INTEREST PROFILE ===\n")

# Aggregate all activities (searches + visits) by semantic category
all_activities = []

# Add search queries
for idx, row in search_queries_df.iterrows():
    all_activities.append({
        'activity_type': 'search',
        'text': row['search_query'],
        'timestamp': row['timestamp'],
        'semantic_category': row['semantic_category'],
        'topic': row['topic'],
        'items': row['items'],
    })

# Add page visits
for idx, row in page_visits_df.iterrows():
    all_activities.append({
        'activity_type': 'page_visit',
        'text': row['title'],
        'timestamp': row['timestamp'],
        'semantic_category': row['semantic_category'],
        'topic': row['topic'],
        'items': row['items'],
    })

# Create activity dataframe
activities_df = pd.DataFrame(all_activities).sort_values('timestamp')

# Generate category-based summary
print("CONSOLIDATED INTERESTS BY SEMANTIC CATEGORY:")
print("=" * 80)

category_summary = activities_df.groupby('semantic_category').agg({
    'activity_type': 'count',
    'topic': lambda x: ', '.join(list(set(x.dropna()))[:5]),
    'items': lambda x: ', '.join(list(set([item for sublist in x for item in sublist]))[:8])
}).rename(columns={'activity_type': 'activity_count', 'topic': 'main_topics', 'items': 'key_entities'})

for category in category_summary.index:
    row = category_summary.loc[category]
    count = row['activity_count']
    topics = row['main_topics']
    items = row['key_entities']
    
    pct = (count / len(activities_df)) * 100
    print(f"\n{category.upper()}")
    print(f"  Activity Count: {int(count)} ({pct:.1f}%)")
    print(f"  Main Topics: {topics if topics else 'N/A'}")
    print(f"  Key Entities: {items if items else 'N/A'}")

# Temporal category trends
print("\n\n" + "=" * 80)
print("CATEGORY TRENDS BY TIME OF DAY:")
print("=" * 80)

activities_df['hour'] = activities_df['timestamp'].dt.hour
peak_categories_by_hour = activities_df.groupby(['hour', 'semantic_category']).size().reset_index(name='count')

for hour in range(0, 24, 3):
    hour_data = peak_categories_by_hour[peak_categories_by_hour['hour'].between(hour, min(hour+2, 23))]
    if len(hour_data) > 0:
        top_cat = hour_data.nlargest(1, 'count').iloc[0]
        print(f"{hour:02d}:00-{hour+2:02d}:59 → {top_cat['semantic_category']:30s} ({int(top_cat['count'])} activities)")

=== COMPREHENSIVE USER INTEREST PROFILE ===



KeyError: 'semantic_category'

## Section 10: Comprehensive User Interest Profile

**Objective:** Combine search and page visit data to create a holistic user profile with consolidated insights.

**What this section does:**
- Aggregates all activities (searches + visits) into single dataframe
- Groups by semantic category and provides activity counts
- Shows main topics and key entities per category
- Analyzes temporal trends showing peak interest times
- Displays category preferences across hours of the day

In [ ]:
# Export structured data to JSON for further use
import json

# Create a clean structured output format
structured_output = {
    'metadata': {
        'total_activities': len(activities_df),
        'total_searches': len(search_queries_df),
        'total_page_visits': len(page_visits_df),
        'total_search_clusters': len(search_clusters),
        'date_range': {
            'start': str(activities_df['timestamp'].min()),
            'end': str(activities_df['timestamp'].max())
        }
    },
    'semantic_categories': {},
    'search_clusters_analysis': [],
    'top_entities': {}
}

# Add category-level insights
for category in activities_df['semantic_category'].unique():
    cat_data = activities_df[activities_df['semantic_category'] == category]
    items = []
    for item_list in cat_data['items']:
        items.extend(item_list)
    
    structured_output['semantic_categories'][category] = {
        'activity_count': int(cat_data.shape[0]),
        'search_count': int(cat_data[cat_data['activity_type'] == 'search'].shape[0]),
        'visit_count': int(cat_data[cat_data['activity_type'] == 'page_visit'].shape[0]),
        'topics': list(set(cat_data['topic'].dropna())),
        'top_entities': list(dict.fromkeys(items))[:10],
    }

# Add cluster analysis
for i, cluster in enumerate(search_clusters[:5], 1):
    context = extract_cluster_context(cluster)
    structured_output['search_clusters_analysis'].append({
        'cluster_id': i,
        'size': context['cluster_size'],
        'dominant_category': context['dominant_category'],
        'category_distribution': context['category_distribution'],
        'topics': context['topics'],
        'entities': context['items'],
        'searches': cluster
    })

# Collect top entities across all categories
all_entities = []
for item_list in activities_df['items']:
    all_entities.extend(item_list)

from collections import Counter
entity_counts = Counter(all_entities)
structured_output['top_entities'] = dict(entity_counts.most_common(20))

# Save to JSON
output_file = 'structured_user_interests.json'
with open(output_file, 'w') as f:
    json.dump(structured_output, f, indent=2, default=str)

print(f"✓ Structured output saved to '{output_file}'")
print(f"\nGenerated JSON contains:")
print(f"  • Metadata about the analysis")
print(f"  • Semantic categories with topics and entities")
print(f"  • Search cluster analysis")
print(f"  • Top entities by frequency")

# Display the structured output
print("\n\n=== STRUCTURED USER INTEREST PROFILE (JSON FORMAT) ===\n")
print(json.dumps(structured_output, indent=2, default=str)[:2000] + "\n...[truncated]")

NameError: name 'activities_df' is not defined

## Section 11: Summary and Generated Files

**Objective:** Provide a summary of all output files generated by this analysis and usage instructions.

**What this section does:**
- Lists all generated files with descriptions and sizes
- Shows when files were created and updated
- Provides next steps and usage examples
- Explains how to load and use the structured output

In [ ]:
# Display summary of all generated files
import os

print("=" * 80)
print("GENERATED FILES SUMMARY")
print("=" * 80)

files_to_check = [
    ('structured_user_interests.json', 'Complete analysis with categories, topics, entities'),
    ('openai_extraction_cache.json', 'Cached API responses (reusable for future runs)'),
]

print("\n📁 Output Files:\n")

for filename, description in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        file_size_mb = file_size / (1024 * 1024)
        
        print(f"✓ {filename}")
        print(f"  Description: {description}")
        print(f"  Size: {file_size_mb:.2f} MB")
        print()
    else:
        print(f"⚠ {filename} - Not found")
        print(f"  Description: {description}")
        print()

print("=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("""
1. Use 'structured_user_interests.json' for analysis and recommendations
2. The cache file will speed up future runs (no re-processing needed)
3. To re-run with fresh data, delete 'openai_extraction_cache.json'

Example: Load the structured data
    import json
    with open('structured_user_interests.json') as f:
        data = json.load(f)
    
    # Access categories
    categories = data['semantic_categories']
    
    # Access top entities
    entities = data['top_entities']
""")

## Section 12: Advanced Visualization and Insights

**Objective:** Create detailed tables and visualizations showing the complete extraction results for all activities.

**What this section does:**
- Creates comprehensive tables of search queries with extraction results
- Shows page visits with extracted categories and entities
- Builds category-to-topic-to-item hierarchical mapping
- Displays activity counts and key entities per category
- Provides detailed view of all extracted structured information

In [ ]:
# Create detailed recommendation table
print("=== DETAILED ENTITY EXTRACTION & CATEGORIZATION TABLE ===\n")

# Create a comprehensive table with all information
detailed_table = []

for idx, row in search_queries_df.head(30).iterrows():
    detailed_table.append({
        'Activity': 'Search',
        'Query/Title': row['search_query'][:50],
        'Category': row['semantic_category'],
        'Topic': row['topic'][:30],
        'Entities': ', '.join(row['items'][:2]) if row['items'] else '-',
        'Entity Count': len(row['items'])
    })

detailed_df = pd.DataFrame(detailed_table)
print("Search Queries - Structured Extraction (OpenAI GPT-4o-mini):")
print(detailed_df.to_string())

print("\n\n" + "=" * 100)
print("PAGE VISITS - STRUCTURED EXTRACTION:\n")

# Similar for page visits
visit_table = []
for idx, row in page_visits_df.head(20).iterrows():
    visit_table.append({
        'Activity': 'Page Visit',
        'Query/Title': row['title'][:50],
        'Category': row['semantic_category'],
        'Topic': row['topic'][:30],
        'Entities': ', '.join(row['items'][:2]) if row['items'] else '-',
    })

visit_df = pd.DataFrame(visit_table)
print(visit_df.to_string())

# Category to Item mapping
print("\n\n" + "=" * 100)
print("CATEGORY → TOPIC → ITEM MAPPING:\n")

category_mapping = activities_df.groupby('semantic_category').apply(
    lambda x: {
        'topics': list(set([t for t in x['topic'] if pd.notna(t)][:5])),
        'items': list(dict.fromkeys([item for items in x['items'] for item in items][:8])),
        'activity_count': len(x)
    }
).to_dict()

for category, info in category_mapping.items():
    print(f"\n{'█' * 3} {category}")
    print(f"    Activity Count: {info['activity_count']}")
    if info['topics']:
        print(f"    Topics: {', '.join(info['topics'])}")
    if info['items']:
        print(f"    Key Items/Entities: {', '.join(info['items'])}")

# Search History Information Extraction & Analysis - Complete Reference

## Executive Summary
This notebook demonstrates a sophisticated approach to analyzing Google search history data. Unlike simple keyword matching or embedding-based similarity, this analysis performs deep semantic understanding through:

1. **Named Entity Recognition (NER)** - Extracts specific entities (brands, products, locations, people)
2. **Semantic Categorization** - Classifies searches and visits into 11+ semantic categories
3. **Behavior Analysis** - Identifies patterns, clusters, and temporal trends
4. **Web Content Extraction** - Analyzes actual pages visited, not just search queries
5. **Comprehensive Profiling** - Combines all signals for holistic user understanding

## Key Features:
- **Dual Processing**: Analyzes both search queries (intent) and page visits (behavior)
- **API Optimization**: Implements caching to reduce costs and improve efficiency
- **Fallback Strategies**: Handles failures gracefully (e.g., scraping fallback to URL parsing)
- **Structured Output**: Generates machine-readable JSON for downstream integration
- **Cost Transparency**: Tracks and reports API usage and estimated costs

## Data Flow:
```
Raw Search History JSON
    ↓
Parse & Classify (searches/visits/other)
    ↓
├─→ Search Queries → Temporal Analysis → NER Extraction
│
└─→ Page Visits → Web Scraping → NER Extraction
    ↓
Combine Results → Cluster Analysis → User Profile
    ↓
Export to Structured JSON
```